In [13]:
import pandas as pd
from analysis import analysis_data
import os.path as osp
from matplotlib import pyplot as plt
import csv

known_data_path = osp.join(osp.dirname(analysis_data.__file__), "known_performance.csv")
df_known = pd.read_csv(known_data_path)

pred_data_path = osp.join(osp.dirname(analysis_data.__file__), "pred_performance.csv")
df_pred = pd.read_csv(pred_data_path)

pred_data_path = osp.join(osp.dirname(analysis_data.__file__), "pred_consumption_predictions.csv")
df_predictions = pd.read_csv(pred_data_path)

# historic_consumptions_data_path = osp.join(osp.dirname(data.__file__), "historic_consumptions.csv")
# df_historic_consumptions = pd.read_csv(historic_consumptions_data_path)


In [16]:
df_pred.head(10)

,0,0.2856653339276576,0.24005842590508153,0.4840943629823778,0.9128833333333334,0.0,0.1755839986604004,0.22
0,1,-0.031749,0.240058,-0.428789,9.918213e-08,0.0,0.175584,0.22
1,2,-0.075234,0.240058,-0.428789,9.918213e-08,0.0,0.175584,0.22
2,3,-0.172681,0.240058,1.503286,1.932075e+00,0.0,0.175584,0.22
3,4,-0.033845,0.240058,-0.138706,2.900833e-01,0.0,0.175584,0.22
4,0,0.118720,0.502107,2.661560,8.333500e-01,0.0,0.170252,0.22
5,1,-0.174060,0.205210,-0.203186,9.918213e-08,0.0,0.170252,0.22
6,2,-0.077015,0.157479,-0.481484,9.974158e-08,0.0,0.170252,0.22
7,3,0.000000,0.050517,-0.675328,4.298000e-01,0.0,0.170252,0.22
8,4,-0.000775,0.202909,0.074998,2.916000e-01,0.0,0.170252,0.22
9,0,0.000000,0.610266,1.569085,8.093000e-01,0.0,0.158084,0.22


In [11]:
df_predictions.head()

,0,72,-1.8324319522090324,-0.004221499770362347
0,1,72,2.360010,-0.008032
1,2,72,2.081728,-0.009476
2,3,72,2.931047,-0.009840
3,4,72,1.246601,-0.004545
4,0,73,-0.764007,-0.004221


In [ ]:
df_pred = pd.concat([df_pred, df_predictions], axis=1)
df_known.head()

In [9]:
df_pred.head()

,0,0.2856653339276576,0.24005842590508153,0.4840943629823778,0.9128833333333334,0.0,0.1755839986604004,0.22,0,72,-1.8324319522090324,-0.004221499770362347,0,72,-1.8324319522090324,-0.004221499770362347,0,72,-1.8324319522090324,-0.004221499770362347
0,1,-0.031749,0.240058,-0.428789,9.918213e-08,0.0,0.175584,0.22,1,72,2.360010,-0.008032,1,72,2.360010,-0.008032,1,72,2.360010,-0.008032
1,2,-0.075234,0.240058,-0.428789,9.918213e-08,0.0,0.175584,0.22,2,72,2.081728,-0.009476,2,72,2.081728,-0.009476,2,72,2.081728,-0.009476
2,3,-0.172681,0.240058,1.503286,1.932075e+00,0.0,0.175584,0.22,3,72,2.931047,-0.009840,3,72,2.931047,-0.009840,3,72,2.931047,-0.009840
3,4,-0.033845,0.240058,-0.138706,2.900833e-01,0.0,0.175584,0.22,4,72,1.246601,-0.004545,4,72,1.246601,-0.004545,4,72,1.246601,-0.004545
4,0,0.118720,0.502107,2.661560,8.333500e-01,0.0,0.170252,0.22,0,73,-0.764007,-0.004221,0,73,-0.764007,-0.004221,0,73,-0.764007,-0.004221


In [5]:
# df_known = pd.concat([df_known, df_historic_consumptions], axis=1)

In [6]:
col_names = ["agent_id",
            "action",
            "electrical_storage_soc",
            "net_electricity_consumption",
            "non_shiftable_load",
            "solar_generation",
            "carbon_intensity",
            "electricity_pricing"]
# historic_col_names = []
# for i in range(10):
#     historic_col_names.append(f"consumption_{i}hours_ago")
#
df_known.columns = col_names # + historic_col_names
pred_col_names = col_names + ["predicted_load", "predicted_solar"]
df_pred.columns = pred_col_names
df_known.head()

ValueError: Length mismatch: Expected axis has 12 elements, new values have 10 elements

In [ ]:
building_num_known = 0
building_num_pred = 0
df_known_building = df_known.loc[df_known["agent_id"] == building_num_known]
df_pred_building = df_pred.loc[df_pred["agent_id"] == building_num_pred]

In [ ]:
exclude_columns = ["predicted_solar", "agent_id", "carbon_intensity", "electricity_pricing"]
df_known_building = df_known_building[[x for x in col_names if x not in exclude_columns]]
df_pred_building = df_pred_building[[x for x in pred_col_names if x not in exclude_columns]]

In [ ]:
left_bound = 600
window_width = 24
right_bound = left_bound + window_width

df_known_building[left_bound:right_bound].plot(kind='line')
plt.legend(loc='upper left')
plt.show()

shift = 72
df_pred_building[left_bound + shift:right_bound + shift].plot(kind='line')
plt.legend(loc='upper left')
plt.show()